## combining individual coverage data into a mean-normalized coverage

## Plan:

+ for the PanelofNormals, create a coverage file for each PON bam and each chromosome
+ Pon bams should optimally not contain CNV or there should be a lot of them to reduce the std
+ put all the files into a big matrix and normalize coverages and produce an average coverage (+ std) for the exonic space
+ compare the tumor samples against that PONcoverage to get differences in CNV

### have to look at the Kenichi-CNV-graphs to identify the samples from FDAH1 without CNVs in the normals
+ [1, 2, 4, 6, 7, 8, 9, 10, 11, 12, 13, 14, 15, 16]

In [6]:
# set the paths
home = '/Users/mahtin'
home = '/Users/martinscience'
testdata = f"{home}/Dropbox/Icke/Work/somVar/testdata"
tooldata = f"{home}/Dropbox/Icke/Work/somVar/tooldata"
shell_path = "../shell"
static_path = f"{home}/Dropbox/Icke/Work/static"
bed_path = f"{static_path}/bed_files/SureSelect/hg38"

cnvdata = f"{tooldata}/myCNVdata/"
bedCov_path = f"{cnvdata}/bedCov"
bedCov_path = f"{home}/mount/scratch/develop/PONcoverage/bedCov"
output_path = f"{cnvdata}/output"

In [7]:
bedCov_path

'/Users/martinscience/mount/scratch/develop/PONcoverage/bedCov'

In [8]:
test = pd.read_csv(f'{bedCov_path}/009_B.chr5.bedCov', sep='\t', compression="gzip")
test[:13]

,Chr,Pos,Coverage,ExonPos
0,chr5,140300,184.30,33
1,chr5,140350,242.62,83
2,chr5,140400,288.67,133
3,chr5,140450,347.07,183
4,chr5,140500,406.70,233
5,chr5,140550,480.91,283
6,chr5,140600,544.79,333
7,chr5,140650,523.75,383
8,chr5,140700,384.47,433
9,chr5,140750,187.23,483


### get all the normal samples (without CNV) into df for normalization and averaging

In [9]:
# try with pd merge
import os
samples = [1, 2, 4, 6, 8, 9, 10, 11, 12, 13, 14, 15, 16]
cov_df = pd.DataFrame(columns=['ExonPos', 'Pos'])
for sample in [f"{str(s).zfill(3)}_B" for s in samples]:
    file = os.path.join(bedCov_path, f"{sample}.chr7.bedCov")
    print(f"Reading {sample} from {file}.")
    df = pd.read_csv(file, sep='\t', compression='gzip').loc[:,['Pos', 'ExonPos', 'Coverage']].rename(columns={'Coverage':sample})
    cov_df = cov_df.merge(df, on=['ExonPos', 'Pos'], how='outer')

cov_df = cov_df.fillna(0).sort_values('ExonPos').set_index(['Pos', 'ExonPos'])
cov_df

Reading 001_B from /Users/martinscience/mount/scratch/develop/PONcoverage/bedCov/001_B.chr7.bedCov.
Reading 002_B from /Users/martinscience/mount/scratch/develop/PONcoverage/bedCov/002_B.chr7.bedCov.
Reading 004_B from /Users/martinscience/mount/scratch/develop/PONcoverage/bedCov/004_B.chr7.bedCov.
Reading 006_B from /Users/martinscience/mount/scratch/develop/PONcoverage/bedCov/006_B.chr7.bedCov.
Reading 008_B from /Users/martinscience/mount/scratch/develop/PONcoverage/bedCov/008_B.chr7.bedCov.
Reading 009_B from /Users/martinscience/mount/scratch/develop/PONcoverage/bedCov/009_B.chr7.bedCov.
Reading 010_B from /Users/martinscience/mount/scratch/develop/PONcoverage/bedCov/010_B.chr7.bedCov.
Reading 011_B from /Users/martinscience/mount/scratch/develop/PONcoverage/bedCov/011_B.chr7.bedCov.
Reading 012_B from /Users/martinscience/mount/scratch/develop/PONcoverage/bedCov/012_B.chr7.bedCov.
Reading 013_B from /Users/martinscience/mount/scratch/develop/PONcoverage/bedCov/013_B.chr7.bedCov.


,,001_B,002_B,004_B,006_B,008_B,009_B,010_B,011_B,012_B,013_B,014_B,015_B,016_B
Pos,ExonPos,,,,,,,,,,,,,
35350,967,19.60,11.43,73.63,4.76,34.20,2.49,2.12,9.74,3.84,6.08,5.74,2.61,0.00
35400,1017,28.99,9.91,97.54,5.52,55.41,2.36,0.00,14.45,2.59,7.58,5.34,3.49,0.46
35450,1067,30.32,4.63,94.73,3.81,63.48,1.41,1.40,12.51,0.75,5.89,2.69,3.22,1.46
36200,1112,51.93,0.49,0.64,56.72,1.97,48.58,1.27,32.37,2.17,0.21,46.69,49.06,26.72
36250,1162,53.62,2.78,1.64,49.94,2.97,39.02,1.72,27.86,4.93,1.19,40.20,40.31,20.82
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
159144650,2423334,475.69,609.91,586.38,170.54,367.27,302.52,296.76,384.28,316.02,238.25,408.12,291.77,147.97
159144700,2423384,384.81,476.35,483.67,156.43,289.89,234.26,228.32,305.63,248.00,203.69,335.25,202.83,111.89
159144750,2423434,306.03,385.24,409.29,138.26,237.72,198.61,196.00,238.32,206.14,170.20,297.01,156.74,98.85


### normalize the coverage to an arbitrary coverage of 250 

In [10]:
# normalize sample coverages to an average coverage of 250
norm_cov = cov_df / cov_df.mean() * 250
norm_cov
# norm_cov.to_csv(f"{bedCov}/all_norm.bedCov", sep='\t')

# normtidy = norm_cov.stack().reset_index().set_index('#').rename(columns={'level_1':'sample', 0:'Coverage'})
# normtidy.to_csv(f"{bedCov}/all_norm_tidy.bedCov", sep='\t')
# normtidy

,,001_B,002_B,004_B,006_B,008_B,009_B,010_B,011_B,012_B,013_B,014_B,015_B,016_B
Pos,ExonPos,,,,,,,,,,,,,
35350,967,10.075265,5.778916,30.537026,2.055259,18.137600,1.346063,0.931566,5.016312,1.829196,2.904132,2.615159,1.294831,0.000000
35400,1017,14.902140,5.010416,40.453368,2.383410,29.386094,1.275787,0.000000,7.442064,1.233754,3.620612,2.432918,1.731403,0.235411
35450,1067,15.585819,2.340891,39.287959,1.645071,33.665931,0.762228,0.615185,6.442922,0.357265,2.813378,1.225571,1.597455,0.747173
36200,1112,26.694313,0.247740,0.265431,24.490397,1.044768,26.261743,0.558061,16.671254,1.033686,0.100307,21.272086,24.338863,13.674283
36250,1162,27.563048,1.405546,0.680167,21.562948,1.575107,21.093726,0.755799,14.348506,2.348421,0.568407,18.315225,19.997953,10.654886
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
159144650,2423334,244.525665,308.365571,243.193008,73.635266,194.777668,163.538546,130.401713,197.912557,150.537100,113.800889,185.940539,144.748269,75.725434
159144700,2423384,197.809332,240.838714,200.595454,67.542891,153.740022,126.638040,100.327939,157.406097,118.135563,97.293192,152.740776,100.624777,57.261058
159144750,2423434,157.312934,194.774233,169.747376,59.697501,126.072228,107.366094,86.125946,122.739983,98.195423,81.296585,135.318532,77.759343,50.587681


### avg_cov has the coverage of the "panel of normals"

In [12]:
chrom = "chr7"
avg_cov = pd.DataFrame(index=norm_cov.index)
avg_cov['meanCov'] = norm_cov.mean(axis=1)
avg_cov['std'] = norm_cov.std(axis=1)

avg_cov = avg_cov.reset_index()
avg_cov['Chr'] = chrom
avg_cov = avg_cov.reset_index().loc[:, ['Chr', 'Pos', 'ExonPos', 'meanCov', 'std']]
avg_cov

,Chr,Pos,ExonPos,meanCov,std
0,chr7,35350,967,6.347794,8.779349
1,chr7,35400,1017,8.469798,12.586051
2,chr7,35450,1067,8.237450,13.232049
3,chr7,36200,1112,12.050226,11.643145
4,chr7,36250,1162,10.836134,10.038445
...,...,...,...,...,...
47076,chr7,159144650,2423334,171.315556,68.092702
47077,chr7,159144700,2423384,136.227220,54.276863
47078,chr7,159144750,2423434,112.845682,43.388823
47079,chr7,159144800,2423484,88.753987,34.236230


### get all tumor samples and normalize

In [6]:
samplesA = [1,2,3,4,5,6,7,8,9,10,11,12,13,14,15,16,21]
cov_dfs = []
for sample in [f"{str(s).zfill(3)}_A" for s in samplesA]:
    file = os.path.join(path, f"{sample}.chr7.bedCov")
    print(f"Reading {sample} from {file}.")
    df = pd.read_csv(file, sep='\t', usecols=['#', 'Coverage']).set_index('#').rename(columns={'Coverage':sample})
    cov_dfs.append(df)
covA_df = pd.concat(cov_dfs, axis=1).fillna(0)
covA_df
covA_tidy = covA_df.stack().reset_index().set_index('#').rename(columns={'level_1':'sample', 0:'Coverage'})
covA_tidy.to_csv(f"{bedCov}/allA_tidy.bedCov", sep='\t')

Reading 001_A from ../bedCov/001_A.chr7.bedCov.
Reading 002_A from ../bedCov/002_A.chr7.bedCov.
Reading 003_A from ../bedCov/003_A.chr7.bedCov.
Reading 004_A from ../bedCov/004_A.chr7.bedCov.
Reading 005_A from ../bedCov/005_A.chr7.bedCov.
Reading 006_A from ../bedCov/006_A.chr7.bedCov.
Reading 007_A from ../bedCov/007_A.chr7.bedCov.
Reading 008_A from ../bedCov/008_A.chr7.bedCov.
Reading 009_A from ../bedCov/009_A.chr7.bedCov.
Reading 010_A from ../bedCov/010_A.chr7.bedCov.
Reading 011_A from ../bedCov/011_A.chr7.bedCov.
Reading 012_A from ../bedCov/012_A.chr7.bedCov.
Reading 013_A from ../bedCov/013_A.chr7.bedCov.
Reading 014_A from ../bedCov/014_A.chr7.bedCov.
Reading 015_A from ../bedCov/015_A.chr7.bedCov.
Reading 016_A from ../bedCov/016_A.chr7.bedCov.
Reading 021_A from ../bedCov/021_A.chr7.bedCov.


,001_A,002_A,003_A,004_A,005_A,006_A,007_A,008_A,009_A,010_A,011_A,012_A,013_A,014_A,015_A,016_A,021_A
#,,,,,,,,,,,,,,,,,
1,349.05,498.71,285.36,439.88,433.74,727.27,255.64,349.36,612.06,625.83,523.42,789.77,620.74,412.63,387.80,396.63,188.50
2,502.48,688.33,385.41,564.85,664.15,1121.34,361.51,452.91,906.54,847.05,720.88,1061.39,861.73,599.49,566.29,551.71,268.71
3,485.78,658.99,369.79,521.23,654.36,1108.93,351.49,402.40,888.03,781.83,682.56,1000.91,819.22,578.60,526.01,521.48,257.41
4,316.60,439.89,260.17,350.51,406.52,698.48,233.04,242.52,564.43,496.54,436.13,655.17,520.61,369.81,309.13,321.66,165.32
5,417.00,393.06,498.26,437.26,407.54,444.09,315.55,271.30,304.76,449.70,423.08,495.19,449.71,449.15,562.17,377.56,168.55
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
48634,0.00,158.91,308.61,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,94.52,0.00,0.00,0.00,222.13,0.00
48635,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,164.12,0.00
48636,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,118.18,0.00


In [7]:
normA_cov = covA_df / covA_df.mean() * 250
normA_cov

,001_A,002_A,003_A,004_A,005_A,006_A,007_A,008_A,009_A,010_A,011_A,012_A,013_A,014_A,015_A,016_A,021_A
#,,,,,,,,,,,,,,,,,
1,176.619490,199.726533,127.421889,343.280611,230.266648,306.333443,151.189863,243.525640,331.952951,324.474278,251.092754,311.238477,291.640626,183.417256,179.668089,240.427761,240.374025
2,254.255154,275.666749,172.097246,440.806704,352.588173,472.319692,213.803189,315.706428,491.665242,439.170282,345.817403,418.280521,404.864317,266.477984,262.362666,334.433603,342.657317
3,245.804945,263.916480,165.122443,406.765829,347.390795,467.092475,207.877190,280.497818,481.626277,405.355648,327.434700,394.446110,384.891957,257.192216,243.700906,316.108889,328.247627
4,160.199773,176.169927,116.173790,273.536616,215.815921,294.206805,137.823837,169.051518,306.120649,257.441251,209.218377,258.194301,244.596814,164.383431,143.220207,194.982713,210.815033
5,211.002227,157.415153,222.488191,341.235973,216.357425,187.055177,186.621660,189.112967,165.287686,233.156101,202.958088,195.148184,211.286055,199.650681,260.453866,228.867976,214.933909
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
48634,0.000000,63.641281,137.803718,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,37.249150,0.000000,0.000000,0.000000,134.649972,0.000000
48635,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,99.485677,0.000000
48636,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,71.637932,0.000000


### get the normalized difCov for every sample

In [14]:
difCov = pd.DataFrame(index=normA_cov.index)

In [31]:
for col in normA_cov.columns:
    difCov[col] = normA_cov[col] / avg_cov['meanCov'] * 2
    difCov[col] = difCov[col].rolling(100).mean()

In [32]:
difCov
difCov_tidy = difCov.stack().reset_index().set_index('#').rename(columns={'level_1':'sample', 0:'Coverage'})
difCov_tidy.to_csv("../bedCov/divCov_tidy.bedCov", sep='\t')

,001_A,002_A,003_A,004_A,005_A,006_A,007_A,008_A,009_A,010_A,011_A,012_A,013_A,014_A,015_A,016_A,021_A
#,,,,,,,,,,,,,,,,,
1,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
5,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
48634,1.367288,2.045857,2.882795,2.009125,0.312594,1.703389,3.137690,2.137259,1.370184,1.838662,1.750852,2.083453,1.233746,2.066507,1.699713,2.354668,1.509845
48635,1.357475,2.029291,2.861164,1.991200,0.292962,1.684336,3.093482,2.111089,1.345403,1.813750,1.725805,2.064383,1.202030,2.037433,1.682546,2.358861,1.497673
48636,1.344678,2.002462,2.824877,1.979764,0.279512,1.656052,3.042178,2.093662,1.309906,1.796468,1.706297,2.032133,1.173202,2.002712,1.661941,2.348511,1.485767


In [26]:
difCov['001_A'].rolling(1000).mean()

#
1             NaN
2             NaN
3             NaN
4             NaN
5             NaN
           ...   
48634    1.423354
48635    1.415789
48636    1.406715
48637    1.396792
48638    1.389984
Name: 001_A, Length: 48638, dtype: float64